# Hint Strategy 1
This notebook demonstrates how an LLM performs when presented with a Spring Security migration issue and provide a hint that can be used to improve the basic rule.
Installing pre requisites and configuring Kai with `GPT-4o-mini`

In [ ]:
%pip uninstall kai -y
%pip install --no-cache-dir git+https://github.com/konveyor/kai.git@main
%pip install python-dotenv

In [23]:
from IPython.display import display, Markdown
from kai.llm_interfacing.model_provider import ModelProvider
from kai.kai_config import KaiConfigModels, SupportedModelProviders
from dotenv import load_dotenv
load_dotenv() 

# Initialize the model provider using GPT-4o via OpenAI
model = ModelProvider.from_config(KaiConfigModels(
    provider=SupportedModelProviders.CHAT_OPENAI,
    args={"model": "gpt-4o-mini"},
))

# Async rendering function for displaying the response
async def rendered_llm_call(prompt: str):
    response = await model.ainvoke_llm(prompt)
    display(Markdown(response.content))
    return response



Below is the snippet we are trying to migrate from springboot2 to springboot3. This code uses the discouraged `configure(AuthenticationManagerBuilder)` method inside a `WebSecurityConfigurerAdapter`, both of which are deprecated in Spring Security 5.7+ and removed in Spring Boot 3.

Our goal is to assess how well the LLM refactors this configuration without a hint, and whether it can replace deprecated patterns with the right approach.

In [24]:
before_code = """\
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.authentication.builders.AuthenticationManagerBuilder;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.config.annotation.web.configuration.WebSecurityConfigurerAdapter;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;

@Configuration
@EnableWebSecurity
public class SecurityConfig extends WebSecurityConfigurerAdapter {
    @Override
    protected void configure(AuthenticationManagerBuilder auth) throws Exception {
        auth.inMemoryAuthentication()
            .withUser("user")
            .password(passwordEncoder().encode("password"))
            .roles("USER");
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }
}
"""

This rule flags the use of `configure(AuthenticationManagerBuilder)`. In Spring Boot 3, developers should instead use a `UserDetailsService` and `PasswordEncoder` bean for authentication configuration.

In [25]:
hint_to_migrate="""\
    
  description: configure(AuthenticationManagerBuilder) override is discouraged since Spring Security 5.7.
  message: |
    Overriding `configure(AuthenticationManagerBuilder)` is **discouraged** in Spring Security 5.7.
    Instead, define a `UserDetailsService` and `PasswordEncoder` bean.

    """

Similar example showing before and after migration

In [26]:
example_before_code="""\

@Configuration
public class SecurityConfiguration extends WebSecurityConfigurerAdapter {
    @Override
    protected void configure(AuthenticationManagerBuilder auth) throws Exception {
        UserDetails user = User.withDefaultPasswordEncoder()
            .username("user")
            .password("password")
            .roles("USER")
            .build();
        auth.inMemoryAuthentication()
            .withUser(user);
    }
}

"""

example_after_code="""\
@Configuration
public class SecurityConfiguration {
    @Bean
    public InMemoryUserDetailsManager userDetailsService() {
        UserDetails user = User.withDefaultPasswordEncoder()
            .username("user")
            .password("password")
            .roles("USER")
            .build();
        return new InMemoryUserDetailsManager(user);
    }
}
"""

In [29]:
prompt = f"""

The following java code needs to be migrated from springboot2 to springboot3.

```java
{before_code}
```
I will provide you with an example before/after code migration
```java
{example_before_code}
{example_after_code}
```
Use the following hint's description and message to migrate. 

{hint_to_migrate}

Also, remove the deprecated and unused APIs and suggest the correct alternatives based on the migration needs. 

Can you provide a refactored version of {before_code} that is compatible with springboot3?
Also, create a step by step process for the above migration? Summarize and create a hint with the changes.

"""

In [30]:

response = await rendered_llm_call(prompt)

To migrate your Spring Boot 2 security configuration to Spring Boot 3, you need to make several changes due to the deprecation of certain methods and classes in Spring Security 5.7 and later. Below is the refactored code and a step-by-step migration process.

### Refactored Code

```java
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.web.builders.HttpSecurity;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.config.annotation.web.configuration.WebSecurityConfigurerAdapter;
import org.springframework.security.config.annotation.authentication.builders.AuthenticationManagerBuilder;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;
import org.springframework.security.core.userdetails.User;
import org.springframework.security.core.userdetails.UserDetails;
import org.springframework.security.core.userdetails.UserDetailsService;
import org.springframework.security.provisioning.InMemoryUserDetailsManager;

@Configuration
@EnableWebSecurity
public class SecurityConfig {

    @Bean
    public UserDetailsService userDetailsService() {
        UserDetails user = User.withUsername("user")
            .password(passwordEncoder().encode("password"))
            .roles("USER")
            .build();
        return new InMemoryUserDetailsManager(user);
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }

    @Bean
    public SecurityFilterChain securityFilterChain(HttpSecurity http) throws Exception {
        http
            .authorizeRequests()
            .anyRequest().authenticated()
            .and()
            .formLogin();
        return http.build();
    }
}
```

### Step-by-Step Migration Process

1. **Remove `WebSecurityConfigurerAdapter`:**
   - The `WebSecurityConfigurerAdapter` class is deprecated. Instead, you will define a `SecurityFilterChain` bean.

2. **Define `UserDetailsService`:**
   - Instead of overriding `configure(AuthenticationManagerBuilder)`, create a `UserDetailsService` bean that returns an `InMemoryUserDetailsManager` with the user details.

3. **Define `PasswordEncoder`:**
   - Keep the `PasswordEncoder` bean as it is, since it is still relevant for encoding passwords.

4. **Create a `SecurityFilterChain` Bean:**
   - Define a `SecurityFilterChain` bean to configure HTTP security settings, such as authorization and login.

5. **Update Imports:**
   - Ensure that you import the necessary classes for the new configuration, such as `SecurityFilterChain`, `HttpSecurity`, and `UserDetailsService`.

### Summary of Changes

- Removed the `WebSecurityConfigurerAdapter` class.
- Created a `UserDetailsService` bean instead of overriding `configure(AuthenticationManagerBuilder)`.
- Defined a `SecurityFilterChain` bean to handle HTTP security configurations.
- Kept the `PasswordEncoder` bean for password encoding.

### Hint for Migration

**Migration Hint:**
- Overriding `configure(AuthenticationManagerBuilder)` is discouraged in Spring Security 5.7. Instead, define a `UserDetailsService` and a `SecurityFilterChain` bean for configuring authentication and authorization.

### Evaluation

| Criteria                                                                 | Result         |
|--------------------------------------------------------------------------|----------------|
| Removed deprecated `configure(AuthenticationManagerBuilder)`            |  Yes           |
| Replaced with `UserDetailsService` and `PasswordEncoder` bean           |  Yes |
| Removed `WebSecurityConfigurerAdapter`                                   |  No           |
| The output compiles                                                      | <todo>          |
| Preserves behavior (in-memory auth, user with role)                      | Yes        |